In [1]:
import sys
import os
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from pathlib import Path
import tensorflow as tf
import pandas as pd
import numpy as np

from utilities import create_model
from utilities import TestConfiguration
from utilities import TestDataset
config = TestConfiguration()

IMAGE_SET_NAME = 'standley_4058_test' # config.IMAGE_SET_NAME
METADATA_FILENAME = 'words_metadata.csv' # config.METADATA_FILE_NAME

2022-07-07 15:38:43.299493: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 15:38:44.080713: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 15:38:44.081511: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 15:38:44.113153: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

## Load the test set locally

In [2]:
# Download test set from Google Cloud Storage
IMAGE_BUCKET = 'fmnh_datasets'

storage_path = f'gs://{IMAGE_BUCKET}/{IMAGE_SET_NAME}/'
!gsutil -m cp -r $storage_path ../images

LOCAL_IMAGE_LOCATION = f'../images/{IMAGE_SET_NAME}'

Copying gs://fmnh_datasets/standley_4058_test/C0358531F-b18p0w3-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0358615F-b17p0w6-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0358756F-b18p0w1-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361323F-b18p0w0-word.jpg...     
Copying gs://fmnh_datasets/standley_4058_test/C0361307F-b18p0w3-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361288F-b18p0w1-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361268F-b19p0w2-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361307F-b18p0w2-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361312F-b18p0w3-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361288F-b18p0w0-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361312F-b18p0w1-word.jpg...
Copying gs://fmnh_datasets/standley_4058_test/C0361323F-b18p0w1-word.jpg...     
Copying gs://fmnh_datasets/standley_4058_test/C0384934F-b17p0w14-word.jpg...  

In [3]:
data_dir = Path('..', 'images', IMAGE_SET_NAME)

test_dataset = TestDataset()
test_dataset.create_dataset(32, data_dir, METADATA_FILENAME)

405


## Model loading

In [14]:
# Load model from Google Cloud Storage
MODEL_BUCKET = 'fmnh_models'
MODEL_NAME = 'iam_train-run_55-standley_train_fine_tuned'
if not os.path.exists('models'):
    os.makedirs('models')

model_uri = f'gs://{MODEL_BUCKET}/{MODEL_NAME}'
!gsutil -m cp -r $model_uri ./models/
prediction_model_filename = Path(f'./models/{MODEL_NAME}')
prediction_model = tf.keras.models.load_model(prediction_model_filename)

Copying gs://fmnh_models/iam_train-run_55-standley_train_fine_tuned/keras_metadata.pb...
Copying gs://fmnh_models/iam_train-run_55-standley_train_fine_tuned/saved_model.pb...
Copying gs://fmnh_models/iam_train-run_55-standley_train_fine_tuned/variables/variables.data-00000-of-00001...
Copying gs://fmnh_models/iam_train-run_55-standley_train_fine_tuned/variables/variables.index...
\ [4/4 files][ 83.0 MiB/ 83.0 MiB] 100% Done                                    
Operation completed over 4 objects/83.0 MiB.                                     


In [ ]:
# OR, load model from local filesystem
MODEL_NAME = 'fine_tuned-prediction'
model_location = Path(f'../transfer_learning/{MODEL_NAME}.model')
prediction_model = tf.keras.models.load_model(model_location)

In [15]:
opt = tf.keras.optimizers.Adam()
prediction_model.compile(optimizer=opt)

## Prediction generation

In [16]:
prediction_results = pd.DataFrame(columns=['label', 'prediction'])
for batch in test_dataset.test_dataset:
    images = batch['image']
    labels = batch['label']
    preds = prediction_model.predict(images)
    pred_texts = test_dataset.decode_batch_predictions(preds)
    pred_texts = [t.replace('[UNK]', '').replace(' ', '') for t in pred_texts]
    orig_texts = []
    for label in labels:
        label = tf.strings.reduce_join(test_dataset.num_to_char(label)).numpy().decode("utf-8")
        orig_texts.append(label)
    orig_texts = [t.replace('[UNK]', '').replace(' ', '') for t in orig_texts]
    new_results = pd.DataFrame(zip(orig_texts, pred_texts), columns=['label', 'prediction'])
    prediction_results = prediction_results.append(new_results, ignore_index=True)
print(prediction_results)

if not os.path.exists('predictions'):
    os.makedirs('predictions')
prediction_results.to_csv(Path('predictions', f'{MODEL_NAME}-predictions.csv'))

1/1 [==============================] - 1s 1s/step
           label   prediction
0             .)            "
1          Maxon         Maan
2    delitescens  dillilconan
3       radicans     Pilicana
4           Damp         trtt
..           ...          ...
400     ropinqua    pospirnou
401    macrosora        aarsu
402          Chr          po.
403   Polypodium    Beleuiaem
404       plesio          Pas

[405 rows x 2 columns]
